In [10]:
import pandas as pd
import numpy as np
from ship_mrcnn import *
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from skimage import measure
from sklearn.neighbors import KDTree
from multiprocessing import Pool
from skimage.morphology import binary_opening, disk, label
import kaggle_util

In [12]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img, min_max_threshold=1e-3, max_mean_threshold=None):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    if np.max(img) < min_max_threshold:
        return '' ## no need to encode if it's all zeros
    if max_mean_threshold and np.mean(img) > max_mean_threshold:
        return '' ## ignore overfilled mask
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def multi_rle_encode(img, **kwargs):
    '''
    Encode connected regions as separated masks
    '''
    labels = label(img)
    if img.ndim > 2:
        return [rle_encode(np.sum(labels==k, axis=2), **kwargs) for k in np.unique(labels[labels>0])]
    else:
        return [rle_encode(labels==k, **kwargs) for k in np.unique(labels[labels>0])]

In [3]:
def masks_as_image(rle, IMG_WIDTH = 768, IMG_HEIGHT = 768):
    # Take the individual ship masks and create a single mask array for all ships
    cnt = len(rle)
    all_masks = np.zeros((IMG_WIDTH, IMG_HEIGHT, cnt), dtype = np.int8)
    for idx, mask in enumerate(rle):
        if isinstance(mask, str):
            all_masks[:,:,idx] += rle_decode(mask)
    return all_masks

def make_total_img(masks, IMG_WIDTH = 768, IMG_HEIGHT = 768):
    tot_mask = np.zeros((IMG_WIDTH, IMG_HEIGHT), dtype = np.int8)
    for i in range(masks.shape[-1]):
        tot_mask += masks[...,i]
    tot_mask = (tot_mask > 0).astype(np.int8)
    return tot_mask

def mask_as_one_img(rle, IMG_WIDTH = 768, IMG_HEIGHT = 768):
    masks = masks_as_image(rle)
    tot_mask = np.zeros((IMG_WIDTH, IMG_HEIGHT), dtype = np.int8)
    for i in range(masks.shape[-1]):
        tot_mask += masks[...,i]
    return tot_mask

In [4]:
sublist = [
    '../result/sub_1_shiponly_1.csv',
    '../result/shiponly_1_sub_1.csv',
    '../result/shiponly_2_sub_2.csv',
]

subs = []
for path in sublist:
    subs.append(pd.read_csv(path))

In [38]:
imgids = subs[0].ImageId.unique()
len(imgids)

88500

In [39]:


out_pred_rows = []
prog = tqdm_notebook(total = len(imgids))
for idx, imgid in enumerate(imgids):
    sub_rle = pd.Series()
    for sub in subs:
        sub_rle = sub_rle.append(sub.loc[sub.ImageId == imgid, 'EncodedPixels'])
        
    one_img = mask_as_one_img(sub_rle)
    one_img = one_img > 1
    cur_rles =  multi_rle_encode(one_img)
    if cur_rles == []:
        cur_rles = ['']
    res = [[imgid, rle] for rle in cur_rles ]
    out_pred_rows += res
    
    prog.update(1)
    
#     plt.figure(figsize=(10,10))
#     plt.imshow(one_img)
#     print(idx)
#     if idx >= 10:
#         break
        
sub = pd.DataFrame(out_pred_rows)
sub.columns = ['ImageId', 'EncodedPixels']

kaggle_util.save_result(sub, '../result/mix_mrcnn.csv', competition = 'airbus-ship-detection', 
                        send = True, index = False)

save result
upload result
cmd: kaggle competitions submit -c airbus-ship-detection -f ../result/mix_mrcnn.csv.7z -m "submit"
